In [2]:
import numpy as np

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import os
import random
import re

# Preparing the data

In [3]:
#nova base de dados

with open('C:/Users/Giovanna/Documents/TCC/keras-projects/dados-iniciais3.csv', encoding="utf8") as myfile:
    novos = myfile.read()
    data_set = re.split('\n|,',novos)
print(data_set)

['Boa', 'Aplicações de Filas', 'Árvores Binárias', 'Listas Estáticas Encadeadas', 'Pilhas', 'Pilhas Estáticas', 'Percursos em Árvores Binárias', 'Filas de Prioridade', 'Listas Estáticas', 'Listas Dinâmicas Simplesmente Encadeadas', 'Listas', 'Filas Estáticas', 'Pilhas Dinâmicas', 'Filas Dinâmicas', 'Listas Dinâmicas Duplamente Encadeadas', 'Aplicações de Pilhas', 'Listas Dinâmicas Simplesmente Encadeadas Circulares', 'Filas', 'Listas Estáticas Sequenciais', 'Listas Estáticas Circulares', 'Árvores Binárias de Busca', 'Recursividade', 'Listas Dinâmicas Duplamente Encadeadas Circulares', 'Listas Dinâmicas', 'Tipos Abstratos de Dados', 'Árvores Binárias de Busca Balanceadas', 'Boa', 'Aplicações de Filas', 'Pilhas', 'Tipos Abstratos de Dados', 'Pilhas Estáticas', 'Recursividade', 'Listas Dinâmicas Duplamente Encadeadas', 'Filas Dinâmicas', 'Listas Dinâmicas Simplesmente Encadeadas', 'Listas Dinâmicas', 'Pilhas Dinâmicas', 'Listas Dinâmicas Duplamente Encadeadas Circulares', 'Filas de Priori

In [4]:
##### Data in list format #####

#itens distintos 
vocab_l = sorted(set(data_set))
print(vocab_l)
print ('{} itens distintos'.format(len(vocab_l)))

['Aplicações de Filas', 'Aplicações de Pilhas', 'Boa', 'Filas', 'Filas Dinâmicas', 'Filas Estáticas', 'Filas de Prioridade', 'Listas', 'Listas Dinâmicas', 'Listas Dinâmicas Duplamente Encadeadas', 'Listas Dinâmicas Duplamente Encadeadas Circulares', 'Listas Dinâmicas Simplesmente Encadeadas', 'Listas Dinâmicas Simplesmente Encadeadas Circulares', 'Listas Estáticas', 'Listas Estáticas Circulares', 'Listas Estáticas Encadeadas', 'Listas Estáticas Sequenciais', 'Percursos em Árvores Binárias', 'Pilhas', 'Pilhas Dinâmicas', 'Pilhas Estáticas', 'Péssima', 'Recursividade', 'Regular', 'Ruim', 'Tipos Abstratos de Dados', 'Árvores Binárias', 'Árvores Binárias de Busca', 'Árvores Binárias de Busca Balanceadas', 'Ótima']
30 itens distintos


In [5]:
# Creating a mapping from unique characters to indices

char2idx = {u:i for i, u in enumerate(vocab_l)}
idx2char = np.array(vocab_l)

text_as_int = np.array([char2idx[c] for c in data_set])

print('{')
for char,_ in zip(char2idx, range(100)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

{
  'Aplicações de Filas':   0,
  'Aplicações de Pilhas':   1,
  'Boa':   2,
  'Filas':   3,
  'Filas Dinâmicas':   4,
  'Filas Estáticas':   5,
  'Filas de Prioridade':   6,
  'Listas':   7,
  'Listas Dinâmicas':   8,
  'Listas Dinâmicas Duplamente Encadeadas':   9,
  'Listas Dinâmicas Duplamente Encadeadas Circulares':  10,
  'Listas Dinâmicas Simplesmente Encadeadas':  11,
  'Listas Dinâmicas Simplesmente Encadeadas Circulares':  12,
  'Listas Estáticas':  13,
  'Listas Estáticas Circulares':  14,
  'Listas Estáticas Encadeadas':  15,
  'Listas Estáticas Sequenciais':  16,
  'Percursos em Árvores Binárias':  17,
  'Pilhas':  18,
  'Pilhas Dinâmicas':  19,
  'Pilhas Estáticas':  20,
  'Péssima':  21,
  'Recursividade':  22,
  'Regular':  23,
  'Ruim':  24,
  'Tipos Abstratos de Dados':  25,
  'Árvores Binárias':  26,
  'Árvores Binárias de Busca':  27,
  'Árvores Binárias de Busca Balanceadas':  28,
  'Ótima':  29,


In [6]:
#char2int das sequências

values = np.array(data_set)
print(values)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

['Boa' 'Aplicações de Filas' 'Árvores Binárias' ... 'Árvores Binárias'
 'Listas' 'Pilhas Estáticas']
[ 2  0 26 ... 26  7 20]


In [7]:
#one-hot encoding das sequências

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
##### Create training examples and targets #####

# The maximum length sentence we want for a single input in characters
seq_length = 25
examples_per_epoch = len(data_set)#(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(onehot_encoded)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
##### Create training examples and targets #####

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#foreach item in sequences apply split_input_target
dataset = sequences.map(split_input_target)

In [10]:
##### Create training batches #####

# Batch size
BATCH_SIZE = 1

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 50

# print(dataset)
# for i in dataset:
#     print(i)

datasetBatch = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
# If your program depends on the batches having the same outer dimension,
# you should set the drop_remainder argument to True to prevent the smaller
# batch from being produced.

# Building the model

In [11]:
##### Building the model #####

# Length of the vocabulary in chars
vocab_size = len(vocab_l)

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(rnn_units,batch_input_shape=(1, 25, 30),
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size = len(vocab_l),
    rnn_units=rnn_units)

In [12]:
 model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (1, 25, 1024)             4321280   
_________________________________________________________________
dense (Dense)                (1, 25, 30)               30750     
Total params: 4,352,030
Trainable params: 4,352,030
Non-trainable params: 0
_________________________________________________________________


In [13]:
##### Testing the model #####

for input_example_batch, target_example_batch in datasetBatch.take(1):
    print('Input:',input_example_batch)
    print()
    
    print('Target:',target_example_batch)
    print()
    
    example_batch_predictions = model(input_example_batch)
    print('Batche Predictions:',example_batch_predictions, "# (batch_size, sequence_length, vocab_size)")

Input: tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
   0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 

# Training the model

In [14]:
##### Attach an optimizer, and a loss function #####

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
print(loss(target_example_batch, example_batch_predictions).numpy())
model.compile(optimizer='adam', loss=loss)

0.69276726


In [15]:
##### Configure checkpoints #####

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_2'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [16]:
##### Execute the training #####

EPOCHS=50
history = model.fit(datasetBatch, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
100/100 [==============================] - 37s 368ms/step - loss: 0.1582
Epoch 2/50
100/100 [==============================] - 30s 300ms/step - loss: 0.1396
Epoch 3/50
100/100 [==============================] - 30s 299ms/step - loss: 0.1391
Epoch 4/50
100/100 [==============================] - 32s 316ms/step - loss: 0.1386
Epoch 5/50
100/100 [==============================] - 32s 323ms/step - loss: 0.1379
Epoch 6/50
100/100 [==============================] - 32s 324ms/step - loss: 0.1369
Epoch 7/50
100/100 [==============================] - 31s 306ms/step - loss: 0.1360
Epoch 8/50
100/100 [==============================] - 31s 309ms/step - loss: 0.1349
Epoch 9/50
100/100 [==============================] - 32s 317ms/step - loss: 0.1341
Epoch 10/50
100/100 [==============================] - 30s 300ms/step - loss: 0.1332
Epoch 11/50
100/100 [==============================] - 30s 299ms/step - loss: 0.1325
Epoch 12/50
100/100 [==============================] - 30s 301ms/step - lo

# Generate text

In [17]:
##### Restore the latest checkpoint #####

#print(tf.train.latest_checkpoint(checkpoint_dir))
model = build_model(vocab_size, rnn_units)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
#model.summary()

In [18]:
def generate_seq(model, start_string):
    
    # Find encoded value of start_string
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(vocab_l)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    index = vocab_l.index(start_string)
    
    # Set shape to (1,1,30)
    conteudo_input = onehot_encoded[index]
    conteudo_input = tf.expand_dims(conteudo_input, 0)
    conteudo_input = tf.expand_dims(conteudo_input, 1)
    
    # Number of characters to generate
    num_generate = 25
    
    # Empty string to store our results
    text_generated = [start_string]
    
    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.2
    model.reset_states()
    for i in range(num_generate):
        predictions = model(conteudo_input) #here
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        conteudo_input = tf.expand_dims(onehot_encoded[predicted_id], 0)
        conteudo_input = tf.expand_dims(conteudo_input, 1)
        text_generated.append(vocab_l[predicted_id])
    return text_generated

In [22]:
print(generate_seq(model, start_string='Ótima'))

['Ótima', 'Filas', 'Listas Estáticas Sequenciais', 'Filas Estáticas', 'Listas Dinâmicas Simplesmente Encadeadas', 'Listas', 'Listas Estáticas', 'Árvores Binárias', 'Aplicações de Filas', 'Listas Dinâmicas Simplesmente Encadeadas Circulares', 'Pilhas Estáticas', 'Listas Estáticas Encadeadas', 'Aplicações de Pilhas', 'Listas Estáticas', 'Listas Dinâmicas Duplamente Encadeadas', 'Recursividade', 'Filas Estáticas', 'Filas Dinâmicas', 'Pilhas Estáticas', 'Listas', 'Percursos em Árvores Binárias', 'Árvores Binárias', 'Listas Dinâmicas Duplamente Encadeadas', 'Tipos Abstratos de Dados', 'Filas', 'Pilhas Estáticas']
